In [1]:
from google.cloud import storage
import tsm
import ndjson
from tsm import save_csv
from datetime import date
from time import strptime
import pandas as pd


In [ ]:

%env GOOGLE_APPLICATION_CREDENTIALS="/Desktop/Coding/NewsScrape/covid_news_scrape_gcp_key.json"
from io import BytesIO

bucket_name = "lexsem_project"

api_key_path = '../NewsScrape/covid_news_scrape_gcp_key.json'
storage_client = storage.Client.from_service_account_json(api_key_path)

jsons = []
bucket = storage_client.get_bucket(bucket_name)
for blob in bucket.list_blobs():
    jsons.append(blob)

In [2]:
# checks on the jsons in the gcp bucket
for index, json in enumerate(jsons):
    print("index {}, name: {}".format(index, json.name))

index 0, name: bth_ids_2014-06-01.json
index 1, name: bth_ids_2014-06-02.json
index 2, name: bth_ids_2014-06-03.json
index 3, name: bth_ids_2014-06-04.json
index 4, name: bth_ids_2014-06-05.json
index 5, name: bth_ids_2014-06-06.json
index 6, name: bth_ids_2014-06-07.json
index 7, name: bth_ids_2014-06-08.json
index 8, name: bth_ids_2014-06-09.json
index 9, name: bth_ids_2014-06-10.json
index 10, name: bth_ids_2014-06-11.json
index 11, name: bth_ids_2014-06-12.json
index 12, name: bth_ids_2014-06-13.json
index 13, name: bth_ids_2014-06-14.json
index 14, name: bth_ids_2014-06-15.json
index 15, name: bth_ids_2014-06-16.json
index 16, name: bth_ids_2014-06-17.json
index 17, name: bth_ids_2014-06-18.json
index 18, name: bth_ids_2014-06-19.json
index 19, name: bth_ids_2014-06-20.json
index 20, name: bth_ids_2014-06-21.json
index 21, name: bth_ids_2014-06-22.json
index 22, name: bth_ids_2014-06-23.json
index 23, name: bth_ids_2014-06-24.json
index 24, name: bth_ids_2014-06-25.json
index 25, 

In [3]:
# extracts user, full_text, and date data from gcp
tweet_data_to_use = []

counter = 1
for index, json in enumerate(jsons):
    content = ""
    
    print("working on json #{}, name: {}".format(index, json.name))    
    content = json.download_as_string()
    json_data = ndjson.loads(content)

    print("length of content string: {}".format(len(content)))
    content = ""
    print("length of json obj: {}".format(len(json_data)))
    print("\n")

    for item in json_data:
        rawtime = item['created_at']
        time_prep = rawtime[-4:] + ' ' + rawtime[4:7] + ' ' + rawtime[8:10]
        st = strptime(time_prep, '%Y %b %d')
        finaldate = date(st.tm_year,st.tm_mon,st.tm_mday).isoformat()
        tweet_data_to_use.append([item['user']['screen_name'],item['full_text'],finaldate])
        
    if(len(tweet_data_to_use) > 600000):
        counter += 1
        save_csv('lexsem_data_part{}.csv'.format(counter),tweet_data_to_use,'USE_QUOTES')
        tweet_data_to_use = []

        

working on json #0, name: bth_ids_2014-06-01.json


Forbidden: 403 GET https://storage.googleapis.com/download/storage/v1/b/lexsem_project/o/bth_ids_2014-06-01.json?generation=1602696190959723&alt=media: The project to be billed is associated with a closed billing account.: ('Request failed with status code', 403, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.PARTIAL_CONTENT: 206>)

In [9]:
import pandas as pd
tweet_df = pd.read_csv('merged_0530_fixed.csv', lineterminator = "\n", header = None)
tweet_df

ValueError: Only length-1 line terminators supported

In [8]:
# # had to use this to fix some csv write errors
# tweet_df = tweet_df[[1,2,3]]
# tweet_df.to_csv("merged_0530_fixed.csv", 
#                 index = False, 
#                 line_terminator = "\r\n", 
#                 encoding = 'utf-8', 
#                 header = False)

In [6]:
# check some handles
tweet_df[tweet_df[0] == 'deray']

,0,1,2
1772,deray,RT @Pervaizistan: Cops slowly appear to be pul...,2014-08-22
1886,deray,RT @jaketapper: Retired Philly police Captain ...,2014-08-22
20204,deray,"RT @LeftSentThis: Nelly, T.I., Kevin Hart, And...",2014-08-22
20358,deray,"Just realized that in my dreams, I've been at ...",2014-08-22
34935,deray,RT @emilyBOOMBOOM: My mom asked me why I was c...,2014-08-22
...,...,...,...
24937967,deray,The police have to be tired of being here. #Fe...,2014-08-22
24938032,deray,"Y'all, I'm calling it a night. Being at the li...",2014-08-22
24938643,deray,Car speeds past cops. 4 cars speed to catch up...,2014-08-22
24938970,deray,Police. Just one checkpoint. This is America. ...,2014-08-22


# split df into time periods

In [15]:
## split the df by time
import time
import datetime
import numpy as np
tweet_df['timestamp'] = tweet_df[2].astype('datetime64[ns]').values.astype(np.int64) // 10 ** 9
tweet_df.loc[tweet_df['timestamp'] <= 1405555199, 'time_period'] = 'T1'
tweet_df.loc[(tweet_df['timestamp'] > 1405555199) & (tweet_df['timestamp'] <= 1407542399), 'time_period'] = 'T2'
tweet_df.loc[(tweet_df['timestamp'] > 1407542399) & (tweet_df['timestamp'] <= 1409529599), 'time_period'] = 'T3'
tweet_df.loc[(tweet_df['timestamp'] > 1409529599) & (tweet_df['timestamp'] <= 1416787199), 'time_period'] = 'T4'
tweet_df.loc[(tweet_df['timestamp'] > 1416787199) & (tweet_df['timestamp'] <= 1417564799), 'time_period'] = 'T5'
tweet_df.loc[(tweet_df['timestamp'] > 1417564799) & (tweet_df['timestamp'] <= 1418255999), 'time_period'] = 'T6'
tweet_df.loc[(tweet_df['timestamp'] > 1418255999) & (tweet_df['timestamp'] <= 1428105599), 'time_period'] = 'T7'
tweet_df.loc[(tweet_df['timestamp'] > 1428105599) & (tweet_df['timestamp'] <= 1429401599), 'time_period'] = 'T8'
tweet_df.loc[tweet_df['timestamp'] > 1429401599, 'time_period'] = 'T9'
tweet_df

,0,1,2,timestamp,time_period
0,colocha_rachel,RT @Show_Me15: How to Support Ferguson- writte...,2014-08-22,1408665600,T3
1,IceyTexasSoulja,No disrespect but they taking that mike brown ...,2014-08-22,1408665600,T3
2,KhristiLauren,RT @HuffingtonPost: Why are so many celebritie...,2014-08-22,1408665600,T3
3,OGkaizo,FBI holding terrorism drills in indiana. Next ...,2014-08-22,1408665600,T3
4,freedomgirl2011,RT @jasonkCNN: A line of roses lines the stree...,2014-08-22,1408665600,T3
...,...,...,...,...,...
24953908,EliToYou,RT @Crewof42: #Ferguson: #chart Killed by Poli...,2014-08-22,1408665600,T3
24953909,alaskaHQ,RT @KoranAddo: Members of the Outcasts Motorcy...,2014-08-22,1408665600,T3
24953910,BrendanWJBF,RT @CodyAlcorn: RT @LakishaJackson: Missouri S...,2014-08-22,1408665600,T3
24953911,SouthpawTrey,RT @LakishaJackson: Missouri State Highway Pat...,2014-08-22,1408665600,T3


In [16]:
for i in range(1,10):
    to_match = "T{}".format(i)
    
    to_write = tweet_df[tweet_df['time_period'] == to_match]
    to_write = to_write[[0,1,2]]
    
    to_write.to_csv("segmented_tweets_{}.csv".format(to_match), 
                    index = False, 
                    line_terminator='\r\n', #tsm needs the line terminator to be in windows form it seems
                    encoding = 'utf-8')
    

In [18]:
for i in range(2,10):
    #print("segmented_tweets_T{}.csv".format(i))
    
    csv_name = "segmented_tweets_T{}.csv".format(i)
    prefix = 'lexsem_all_no_isolates_T{}'.format(i)
    print(type(csv_name))
    print(prefix)
    edge_list = tsm.t2e(csv_name,'ALL_NO_ISOLATES', save_prefix=prefix)
    

<class 'str'>
lexsem_all_no_isolates_T2
Edge list created.
Data saved to file "lexsem_all_no_isolates_T2_edgelist.csv".
<class 'str'>
lexsem_all_no_isolates_T3
Edge list created.
Data saved to file "lexsem_all_no_isolates_T3_edgelist.csv".
<class 'str'>
lexsem_all_no_isolates_T4
Edge list created.
Data saved to file "lexsem_all_no_isolates_T4_edgelist.csv".
<class 'str'>
lexsem_all_no_isolates_T5
Edge list created.
Data saved to file "lexsem_all_no_isolates_T5_edgelist.csv".
<class 'str'>
lexsem_all_no_isolates_T6
Edge list created.
Data saved to file "lexsem_all_no_isolates_T6_edgelist.csv".
<class 'str'>
lexsem_all_no_isolates_T7
Edge list created.
Data saved to file "lexsem_all_no_isolates_T7_edgelist.csv".
<class 'str'>
lexsem_all_no_isolates_T8
Edge list created.
Data saved to file "lexsem_all_no_isolates_T8_edgelist.csv".
<class 'str'>
lexsem_all_no_isolates_T9
Edge list created.
Data saved to file "lexsem_all_no_isolates_T9_edgelist.csv".


# full set edge list and partition creation

In [2]:
import tsm

# saves the edgelist
edge_list = tsm.t2e("merged_0530_fixed.csv",'ALL_NO_ISOLATES', save_prefix='lexsem_all_no_isolates_0530')


Edge list created.
Data saved to file "lexsem_all_no_isolates_0530_edgelist.csv".


In [ ]:
# saves a community csv and a louvain obj
partitioned_network = tsm.get_top_communities("lexsem_all_no_isolates_edgelist.csv", top_comm = 1.0,
                                              save_prefix = 'lexsem_all_no_isolates_ALLCOMMS')

Data loaded from file "lexsem_all_no_isolates_edgelist.csv".
Non-directed network created.


## testing seed communities

In [3]:
seed_comms = pd.read_csv('seed_communities.csv')
seed_comms['community'] = seed_comms['community'].astype(str)
seed_comms_dict = {}
for index, row in seed_comms.iterrows():
    seed_comms_dict[row['name']] = row['community']

# change string values to int values
seed_comms_dict_test = dict([a, int(x)] for a, x in seed_comms_dict.items())
seed_comms_dict_test

{'deray': 1,
 'nettaaaaaaaa': 1,
 'shaunking': 1,
 'djolder': 1,
 'bassem_masri': 1,
 'handsupunited_': 1,
 're_invent_ed': 1,
 'brownblaze': 1,
 'awkward_duck': 1,
 'anonymouspress': 2,
 'anonyops': 2,
 'anonypress': 2,
 'bipartisanism': 2,
 'crypt0nymous': 2,
 'occupythemob': 2,
 'youranoncentral': 2,
 'youranonglobal': 2,
 'youranonlive': 2,
 'youranonnews': 2,
 'pharrell': 3,
 '50cent': 3,
 'nas': 3,
 'kanyewest': 3,
 'vibemagazine': 3,
 'XXL': 3,
 'thesource': 3,
 'kerrywashington': 3,
 'ijessewilliams': 3,
 'iamrashidajones': 3,
 'rsherman_25': 3,
 'kobebryant': 3,
 'larryelder': 4,
 'waynedupreeshow': 4,
 'patdollard': 4,
 'seanhannity': 4,
 'gopblackchick': 4,
 'amymek': 4,
 'foxnews': 4,
 'washtimes': 4,
 'theblaze': 4,
 'CNN': 5,
 'nytimes': 5,
 'AP': 5,
 'latimes': 5,
 'CBSNews': 5,
 'Reuters': 5,
 'mashable': 5,
 'BuzzFeed': 5,
 'thedailybeast': 5,
 'HuffPost': 5,
 'bestcomedyvine': 6,
 'commonblackgiri': 6,
 'drugfambeezy': 6,
 'freetopher': 6,
 'hbcufessions': 6,
 'igiveb

In [3]:
edge_list = pd.read_csv("lexsem_all_no_isolates_0530_edgelist.csv")


In [4]:
edge_list = edge_list.dropna().rename(columns={0: "id1", 1: "id2"})

In [5]:
# 36057000 rows as of 05_30
edge_list

,id1,id2
0,colocha_rachel,show_me15
1,colocha_rachel,organizemo
2,colocha_rachel,obs_stl
3,khristilauren,huffingtonpost
4,freedomgirl2011,jasonkcnn
...,...,...
36057121,paulstewartii,liberalmmama
36057122,paulstewartii,tvitin
36057123,emillyorr,omarvaid
36057124,sharanlouise,adybarkan


In [6]:
list1 = edge_list['id1'].unique().tolist()
list2 = edge_list['id2'].unique().tolist()

In [19]:
len(list2)

762195

In [7]:
resultList= list(set(list1) | set(list2))
len(resultList)

4736383

In [8]:
# generate list of unique nodes
nodes = pd.Series(resultList)
nodes.to_csv("lexsem_nodes_0530.csv", encoding = 'utf-8')

In [12]:
# generate list of all nodes 
edge_list.to_csv("lexsem_all_no_isolates_edgelist.csv", encoding = 'utf-8')


In [4]:
import tsm
# init part is a seed community that is formatted as a dict
# saves a community csv and a louvain obj
partitioned_network = tsm.get_top_communities("lexsem_all_no_isolates_edgelist.csv", 
                                              init_part = seed_comms_dict_test, 
                                              save_prefix = 'lexsem_all_no_isolates_ALLCOMMS_seeded')

Data loaded from file "lexsem_all_no_isolates_edgelist.csv".
Non-directed network created.


KeyError: 'frogsarelovely'

# time segmented communities

In [22]:
# diff time period partition network
for i in range(2,10):
    edge_list_name = "lexsem_all_no_isolates_T{}_edgelist.csv".format(i)
    prefix = 'lexsem_all_no_isolates_communities_T{}'.format(i)
    
    partitioned_network = tsm.get_top_communities(edge_list_name,
                                              save_prefix = prefix)

FileNotFoundError: [Errno 2] No such file or directory: 'lexsem_all_no_isolates_T2_edgelist.csv'

# Additional Cleaning/grouping of communities

In [24]:
import pandas as pd
df = pd.read_csv('lexsem_all_no_isolates_communities.csv')

df_filtered = df[df['in_degree'] >= 0.001*len(df.index)]
df_filtered

,name,community,in_degree
0,bipartisanism,0,158667
1,deray,0,134719
2,cnn,4,121073
3,antoniofrench,7,103627
4,youranonnews,2,89336
...,...,...,...
913,uberfacts,0,2557
914,larissaliburd,0,2554
915,nationalaction,1,2552
916,cp24,0,2541


In [27]:
for i in range(2,10):
    df = pd.read_csv('lexsem_all_no_isolates_communities_T{}_communities.csv'.format(i))
    
    ## filter proportional to number of nodes?
    df_filtered = df[df['in_degree'] >= 0.001*len(df.index)]
    community_groups_w_cutoff = df_filtered.groupby('community')['name'].apply(list)
    community_groups_w_cutoff.to_csv("lexsem_all_no_isolates_T{}_communities_grouping_w_cutoff.csv".format(i))
    

community
0     [yamiche, fox2now, nettaaaaaaaa, zellieimani, ...
1     [antoniofrench, michaelskolnik, occupythemob, ...
2     [ryanjreilly, pdpj, stltoday, wilw, wesleylowe...
3     [msnbc, slate, marmel, aclu, 140elect, tpm, di...
6     [cnn, cnnbrk, attorneycrump, ijessewilliams, r...
7     [fergusonunity, newsweek, bbcworld, cassfm, rt...
9     [nytimes, mashable, ap, breakingnews, nydailyn...
11    [jackfrombkln, chrislhayes, occupyoakland, cmc...
13    [youranonnews, alicesperi, vicenews, anonymous...
14    [foxnews, abc, jaketapper, thereval, larryelde...
Name: name, dtype: object


In [26]:
community_groups_w_cutoff = df_filtered.groupby('community')['name'].apply(list)
community_groups_w_cutoff

community
0     [bipartisanism, deray, complexmag, michaelskol...
1     [nettaaaaaaaa, theroot, zellieimani, elonjames...
2     [youranonnews, ksdknews, kodacohen, stltoday, ...
4     [cnn, nytimes, cnnbrk, abc, bbcbreaking, ap, a...
5     [foxnews, fox2now, stlcountypd, cbsnews, there...
7     [antoniofrench, ryanjreilly, huffingtonpost, m...
8     [rolandsmartin, attorneycrump, jamalhbryant, c...
9     [jackfrombkln, aurabogado, arthur_affect, bill...
13    [rt_com, rt_america, actualidadrt, independent...
Name: name, dtype: object

In [3]:
community_groups_w_cutoff.to_csv("lexsem_all_no_isolates_communities_grouping_w_cutoff1000.csv")

In [4]:
community_groups_no_cutoff = df.groupby('community')['name'].apply(list)
community_groups_no_cutoff.to_csv('lexsem_all_no_isolates_communities_grouping_no_cutoff.csv')

# make the format for Sandeep

In [3]:
# make a hash id
tweet_df['id'] = tweet_df.apply(lambda x: hash(tuple(x)), axis = 1)

In [4]:
# rename the column names

tweet_df = tweet_df.rename(columns={0: "name", 1: "tweet", 2: "timestamp"})


In [5]:
tweet_df

,name,tweet,timestamp,id
0,colocha_rachel,RT @Show_Me15: How to Support Ferguson- writte...,2014-08-22,5534988613932907243
1,IceyTexasSoulja,No disrespect but they taking that mike brown ...,2014-08-22,3903426521497550846
2,KhristiLauren,RT @HuffingtonPost: Why are so many celebritie...,2014-08-22,1907900934480168215
3,OGkaizo,FBI holding terrorism drills in indiana. Next ...,2014-08-22,4450107586195367042
4,freedomgirl2011,RT @jasonkCNN: A line of roses lines the stree...,2014-08-22,-2363481464566730715
...,...,...,...,...
24953908,EliToYou,RT @Crewof42: #Ferguson: #chart Killed by Poli...,2014-08-22,5864757158306517572
24953909,alaskaHQ,RT @KoranAddo: Members of the Outcasts Motorcy...,2014-08-22,5895076777260891004
24953910,BrendanWJBF,RT @CodyAlcorn: RT @LakishaJackson: Missouri S...,2014-08-22,-7578028274063970307
24953911,SouthpawTrey,RT @LakishaJackson: Missouri State Highway Pat...,2014-08-22,2087636071824131009


In [6]:
df = pd.read_csv('neo4j_community_export.csv')
df

,name,communityId,intermediateCommunityIds
0,0,4,NaN
1,00,4,NaN
2,0000000000agv,2408214,NaN
3,000000000lana,2,NaN
4,000000000xxxxx,1,NaN
...,...,...,...
2906471,zzzzzxx1231,315993,NaN
2906472,zzzzzzzzsma,64973,NaN
2906473,zzzzzzzzzxxxxdd,1,NaN
2906474,zzzzzzzzzzzzoey,283,NaN


In [39]:
len(df['communityId'].unique())

37959

In [7]:
tweet_df = tweet_df.merge(df, on= 'name', how = 'left')

In [8]:
# remove NAs
tweet_df = tweet_df[tweet_df['communityId'].notnull()]

In [17]:
tweet_df['communityId'].isna().sum()/tweet_df['name'].count()

0.0

In [9]:
tweet_df = tweet_df[["id", "timestamp", "communityId", "tweet"]]
tweet_df

,id,timestamp,communityId,tweet
0,5534988613932907243,2014-08-22,1.0,RT @Show_Me15: How to Support Ferguson- writte...
4,-2363481464566730715,2014-08-22,1.0,RT @jasonkCNN: A line of roses lines the stree...
5,-267571337920445537,2014-08-22,1.0,RT @SPZanti: How the hell does anybody shoot *...
8,-755293806203422642,2014-08-22,2408214.0,RT @CNN: Things got heated between @donlemon a...
9,-8746472106330624551,2014-08-22,2.0,"All injustice needs to be recognized, not just..."
...,...,...,...,...
24953900,-7995974308897741711,2014-08-22,2.0,RT @HnstyNgov: Accomplices? @andersoncooper pr...
24953902,5631830937533997302,2014-08-22,9695.0,@JeamesSailor @jchevinsky @jenromnes @jimrattr...
24953903,8507923606824474338,2014-08-22,2.0,RT @bkesling: “Go kill ISIS and leave us alone...
24953906,-2064711876035607383,2014-08-22,2.0,RT @SavingMila7: “@AnomolyAnon: #WeDoNotForgiv...


In [10]:
tweet_df = tweet_df[tweet_df['communityId'].isin([2408214, 2, 201, 223, 1, 4, 43087, 9695, 64973, 2877752])]

In [11]:
tweet_df = tweet_df.rename(columns={"id": "tweetid", "communityId": "source"})
tweet_df['tweetid'] = tweet_df['tweetid'].astype(str)
tweet_df['source'] = tweet_df['source'].astype(int)
tweet_df = tweet_df.drop_duplicates()
tweet_df

,tweetid,timestamp,source,tweet
0,5534988613932907243,2014-08-22,1,RT @Show_Me15: How to Support Ferguson- writte...
4,-2363481464566730715,2014-08-22,1,RT @jasonkCNN: A line of roses lines the stree...
5,-267571337920445537,2014-08-22,1,RT @SPZanti: How the hell does anybody shoot *...
8,-755293806203422642,2014-08-22,2408214,RT @CNN: Things got heated between @donlemon a...
9,-8746472106330624551,2014-08-22,2,"All injustice needs to be recognized, not just..."
...,...,...,...,...
24953900,-7995974308897741711,2014-08-22,2,RT @HnstyNgov: Accomplices? @andersoncooper pr...
24953902,5631830937533997302,2014-08-22,9695,@JeamesSailor @jchevinsky @jenromnes @jimrattr...
24953903,8507923606824474338,2014-08-22,2,RT @bkesling: “Go kill ISIS and leave us alone...
24953906,-2064711876035607383,2014-08-22,2,RT @SavingMila7: “@AnomolyAnon: #WeDoNotForgiv...


In [50]:
tweet_df_test = pd.read_csv("tweet_communities_v3_022121.tsv", sep = '\t')
#tweet_df = tweet_df.dropna().drop_duplicates()
tweet_df_test

,tweetid,time_period,source,tweet
0,5534988613932907243,T3,1,RT @Show_Me15: How to Support Ferguson- writte...
1,-2363481464566730715,T3,1,RT @jasonkCNN: A line of roses lines the stree...
2,-267571337920445537,T3,1,RT @SPZanti: How the hell does anybody shoot *...
3,-755293806203422642,T3,2408214,RT @CNN: Things got heated between @donlemon a...
4,-8746472106330624551,T3,2,"All injustice needs to be recognized, not just..."
...,...,...,...,...
9980532,-7995974308897741711,T3,2,RT @HnstyNgov: Accomplices? @andersoncooper pr...
9980533,5631830937533997302,T3,9695,@JeamesSailor @jchevinsky @jenromnes @jimrattr...
9980534,8507923606824474338,T3,2,RT @bkesling: “Go kill ISIS and leave us alone...
9980535,-2064711876035607383,T3,2,RT @SavingMila7: “@AnomolyAnon: #WeDoNotForgiv...


In [30]:
tweet_df_test[tweet_df_test['tweet'].isnull()]

,tweetid,time_period,source,tweet
5734,Tear gas in #Ferguson &amp; #Palestine Both ma...,NaN,NaN,NaN
5735,v @shawncarrie @FergusonUnity http://t.co/p…,NaN,NaN,NaN
10397,How true haha 😂😂😂😂 #ferguson #mikebrown #darre...,NaN,NaN,NaN
19128,http://t.co/6ZZRriQW6r,NaN,NaN,NaN
20734,How true haha 😂😂😂😂 #ferguson #mikebrown #darre...,NaN,NaN,NaN
...,...,...,...,...
42599891,@Lawrence #Ferguson,NaN,NaN,NaN
42600627,http://t.co/CWQJdpYmi7,NaN,NaN,NaN
42600689,@Lawrence #Ferguson,NaN,NaN,NaN
42601104,@Lawrence #Ferguson,NaN,NaN,NaN


In [28]:
tweet_df_test2 = tweet_df_test.dropna().drop_duplicates()
tweet_df_test2

,tweetid,time_period,source,tweet
0,5534988613932907243,T3,1.0,RT @Show_Me15: How to Support Ferguson- writte...
1,-2363481464566730715,T3,1.0,RT @jasonkCNN: A line of roses lines the stree...
2,-267571337920445537,T3,1.0,RT @SPZanti: How the hell does anybody shoot *...
3,-755293806203422642,T3,2408214.0,RT @CNN: Things got heated between @donlemon a...
4,-8746472106330624551,T3,2.0,"All injustice needs to be recognized, not just..."
...,...,...,...,...
42613784,-7995974308897741711,T3,2.0,RT @HnstyNgov: Accomplices? @andersoncooper pr...
42613785,5631830937533997302,T3,9695.0,@JeamesSailor @jchevinsky @jenromnes @jimrattr...
42613786,8507923606824474338,T3,2.0,RT @bkesling: “Go kill ISIS and leave us alone...
42613787,-2064711876035607383,T3,2.0,RT @SavingMila7: “@AnomolyAnon: #WeDoNotForgiv...


In [33]:
tweet_df['timestamp'].unique()

array(['2014-08-22', '2014-08-23', '2014-08-24', '2014-08-25',
       '2014-08-26', '2014-08-27', '2014-08-28', '2014-08-29',
       '2014-08-30', '2014-08-31', '2014-09-01', '2014-09-02',
       '2014-09-03', '2014-09-04', '2014-09-05', '2014-09-06',
       '2014-09-07', '2014-09-08', '2014-09-09', '2014-09-10',
       '2014-09-11', '2014-09-12', '2014-09-13', '2014-09-14',
       '2014-09-15', '2014-09-16', '2014-09-17', '2014-09-18',
       '2014-09-19', '2014-09-20', '2014-09-21', '2014-09-22',
       '2014-09-23', '2014-09-24', '2014-09-25', '2014-09-26',
       '2014-09-27', '2014-09-28', '2014-09-29', '2014-09-30',
       '2014-10-01', '2014-10-02', '2014-10-03', '2014-10-04',
       '2014-10-05', '2014-10-06', '2014-10-07', '2014-10-08',
       '2014-10-09', '2014-10-10', '2014-10-11', '2014-10-12',
       '2014-10-13', '2014-10-14', '2014-10-15', '2014-10-16',
       '2014-10-17', '2014-10-18', '2014-10-19', '2014-10-20',
       '2014-10-21', '2014-10-22', '2014-10-23', '2014-

In [12]:
import time
import datetime
import numpy as np
tweet_df['ts'] = tweet_df.timestamp.astype('datetime64[ns]').values.astype(np.int64) // 10 ** 9
tweet_df

,tweetid,timestamp,source,tweet,ts
0,5534988613932907243,2014-08-22,1,RT @Show_Me15: How to Support Ferguson- writte...,1408665600
4,-2363481464566730715,2014-08-22,1,RT @jasonkCNN: A line of roses lines the stree...,1408665600
5,-267571337920445537,2014-08-22,1,RT @SPZanti: How the hell does anybody shoot *...,1408665600
8,-755293806203422642,2014-08-22,2408214,RT @CNN: Things got heated between @donlemon a...,1408665600
9,-8746472106330624551,2014-08-22,2,"All injustice needs to be recognized, not just...",1408665600
...,...,...,...,...,...
24953900,-7995974308897741711,2014-08-22,2,RT @HnstyNgov: Accomplices? @andersoncooper pr...,1408665600
24953902,5631830937533997302,2014-08-22,9695,@JeamesSailor @jchevinsky @jenromnes @jimrattr...,1408665600
24953903,8507923606824474338,2014-08-22,2,RT @bkesling: “Go kill ISIS and leave us alone...,1408665600
24953906,-2064711876035607383,2014-08-22,2,RT @SavingMila7: “@AnomolyAnon: #WeDoNotForgiv...,1408665600


In [23]:
tweet_df['timestamp'].describe()

count       27824170
unique           365
top       2015-04-28
freq         9084337
Name: timestamp, dtype: object

In [13]:
tweet_df.loc[tweet_df['ts'] <= 1405555199, 'time_period'] = 'T1'
tweet_df.loc[(tweet_df['ts'] > 1405555199) & (tweet_df['ts'] <= 1407542399), 'time_period'] = 'T2'
tweet_df.loc[(tweet_df['ts'] > 1407542399) & (tweet_df['ts'] <= 1409529599), 'time_period'] = 'T3'
tweet_df.loc[(tweet_df['ts'] > 1409529599) & (tweet_df['ts'] <= 1416787199), 'time_period'] = 'T4'
tweet_df.loc[(tweet_df['ts'] > 1416787199) & (tweet_df['ts'] <= 1417564799), 'time_period'] = 'T5'
tweet_df.loc[(tweet_df['ts'] > 1417564799) & (tweet_df['ts'] <= 1418255999), 'time_period'] = 'T6'
tweet_df.loc[(tweet_df['ts'] > 1418255999) & (tweet_df['ts'] <= 1428105599), 'time_period'] = 'T7'
tweet_df.loc[(tweet_df['ts'] > 1428105599) & (tweet_df['ts'] <= 1429401599), 'time_period'] = 'T8'
tweet_df.loc[tweet_df['ts'] > 1429401599, 'time_period'] = 'T9'


In [14]:
tweet_df

,tweetid,timestamp,source,tweet,ts,time_period
0,5534988613932907243,2014-08-22,1,RT @Show_Me15: How to Support Ferguson- writte...,1408665600,T3
4,-2363481464566730715,2014-08-22,1,RT @jasonkCNN: A line of roses lines the stree...,1408665600,T3
5,-267571337920445537,2014-08-22,1,RT @SPZanti: How the hell does anybody shoot *...,1408665600,T3
8,-755293806203422642,2014-08-22,2408214,RT @CNN: Things got heated between @donlemon a...,1408665600,T3
9,-8746472106330624551,2014-08-22,2,"All injustice needs to be recognized, not just...",1408665600,T3
...,...,...,...,...,...,...
24953900,-7995974308897741711,2014-08-22,2,RT @HnstyNgov: Accomplices? @andersoncooper pr...,1408665600,T3
24953902,5631830937533997302,2014-08-22,9695,@JeamesSailor @jchevinsky @jenromnes @jimrattr...,1408665600,T3
24953903,8507923606824474338,2014-08-22,2,RT @bkesling: “Go kill ISIS and leave us alone...,1408665600,T3
24953906,-2064711876035607383,2014-08-22,2,RT @SavingMila7: “@AnomolyAnon: #WeDoNotForgiv...,1408665600,T3


In [15]:
tweet_df_new = tweet_df[["tweetid", "time_period", "source", "tweet"]]
tweet_df_new

,tweetid,time_period,source,tweet
0,5534988613932907243,T3,1,RT @Show_Me15: How to Support Ferguson- writte...
4,-2363481464566730715,T3,1,RT @jasonkCNN: A line of roses lines the stree...
5,-267571337920445537,T3,1,RT @SPZanti: How the hell does anybody shoot *...
8,-755293806203422642,T3,2408214,RT @CNN: Things got heated between @donlemon a...
9,-8746472106330624551,T3,2,"All injustice needs to be recognized, not just..."
...,...,...,...,...
24953900,-7995974308897741711,T3,2,RT @HnstyNgov: Accomplices? @andersoncooper pr...
24953902,5631830937533997302,T3,9695,@JeamesSailor @jchevinsky @jenromnes @jimrattr...
24953903,8507923606824474338,T3,2,RT @bkesling: “Go kill ISIS and leave us alone...
24953906,-2064711876035607383,T3,2,RT @SavingMila7: “@AnomolyAnon: #WeDoNotForgiv...


In [47]:
tweet_df_new['tweet'] = tweet_df_new['tweet'].str.replace('\n', ' ')
tweet_df_new['tweet'] = tweet_df_new['tweet'].str.replace('\t', ' ')
tweet_df_new['tweet'] = tweet_df_new['tweet'].str.replace('\r', ' ')

/Users/alexfan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/alexfan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/alexfan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [48]:
tweet_df_new[tweet_df_new['tweet'].str.contains("\r")]

,tweetid,time_period,source,tweet


In [49]:
tweet_df_new.to_csv("tweet_communities_v3_022121.tsv", sep = '\t', index = False, encoding = 'utf-8')

In [33]:
tweet_df_new[tweet_df_new['time_period'] == "T6"]

,tweetid,time_period,source,tweet
9400407,-3656759192320548007,T6,1,RT @kodacohen: McCluer North students walk to ...
9400409,-2615873630829356848,T6,2,RT @alexirvine: Cliven Bundy supporters point ...
9400413,-3094009144729580026,T6,2,RT @Lnonblonde: Cops Who Shot #TamirRice In Th...
9400414,4633361979888269309,T6,1,RT @RockawayYouth: #BlackLivesMatter. Period. ...
9400415,-918933322871024872,T6,2408214,RT @KeithOlbermann: VIDEO WORSTS: Police intim...
...,...,...,...,...
12124811,8883198260281297527,T6,1,RT @Nettaaaaaaaa: Spotted this on Facebook jus...
12124815,-1238483596014322431,T6,1,RT @freeandclear1: Black preschoolers more lik...
12124817,-829059684413177875,T6,2408214,TODAY: March for justice for #EricGarner #Mike...
12124820,41513305162981551,T6,2408214,RT @ACLU: Join us &amp; @NYCLU @ 5:30pm 2night...


In [46]:
tweet_df_new.to_csv("tweet_communities_v3_022121.csv", sep = '\t', index = False, encoding = 'utf-8')

In [33]:
tweet_df_test = pd.read_csv("tweet_communities_v3_022121.csv")
#tweet_df = tweet_df.dropna().drop_duplicates()
tweet_df_test

/Users/alexfan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,1,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tweetid,time_period,source,tweet
0,5534988613932907243,T3,1.0,RT @Show_Me15: How to Support Ferguson- writte...
1,-2363481464566730715,T3,1.0,RT @jasonkCNN: A line of roses lines the stree...
2,-267571337920445537,T3,1.0,RT @SPZanti: How the hell does anybody shoot *...
3,-755293806203422642,T3,2408214.0,RT @CNN: Things got heated between @donlemon a...
4,-8746472106330624551,T3,2.0,"All injustice needs to be recognized, not just..."
...,...,...,...,...
34759754,-7995974308897741711,T3,2.0,RT @HnstyNgov: Accomplices? @andersoncooper pr...
34759755,5631830937533997302,T3,9695.0,@JeamesSailor @jchevinsky @jenromnes @jimrattr...
34759756,8507923606824474338,T3,2.0,RT @bkesling: “Go kill ISIS and leave us alone...
34759757,-2064711876035607383,T3,2.0,RT @SavingMila7: “@AnomolyAnon: #WeDoNotForgiv...
